In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

def fetch_wikidata_info(entity_label):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = f"""
    SELECT ?entity ?entityLabel ?description WHERE {{
      ?entity ?label "{entity_label}"@en.
      ?entity rdfs:label ?entityLabel.
      ?entity schema:description ?description.
      FILTER (lang(?entityLabel) = "en" && lang(?description) = "en")
    }} LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if results['results']['bindings']:
        result = results['results']['bindings'][0]
        entity = result['entity']['value']
        entity_label = result['entityLabel']['value']
        description = result['description']['value']
        return entity_label, description
    else:
        return None, None

entity_label, description = fetch_wikidata_info("Albert Einstein")
print(f"Label: {entity_label}, Description: {description}")

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def generate_response_with_kg(input_text, entity_info):
    entity_label, description = entity_info

    # Incorporate the knowledge graph information into the prompt
    if entity_label and description:
        knowledge_prompt = f"{entity_label} is known for: {description}. "
    else:
        knowledge_prompt = ""

    # Combine the knowledge prompt with the input text
    combined_prompt = knowledge_prompt + input_text

    # Tokenize and generate the response
    inputs = tokenizer.encode(combined_prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1)

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
input_text = "Tell me something interesting about Albert Einstein."
entity_info = fetch_wikidata_info("Albert Einstein")
response = generate_response_with_kg(input_text, entity_info)
print(response)